# Test Bed


In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np

from src.sequence.gen_spectra import gen_spectrum, get_precursor
from src.objects import Spectrum
from src.utils import insort_by_index, make_sparse_array, ppm_to_da
from src.scoring import scoring

from src import main

from src.params import OUTPUT_DIRECTORY

from collections import namedtuple

In [2]:
# run hyped search with the params 
parameters = namedtuple('parameteres', 'params')
main.main(parameters('True'))

Loading database...
Done
Loading spectra...
Done
Indexing potential hits...
On protein 279/279 [100%]
Done.
Creating an alignment for 1/9 [11%]Results that made it through the filter: 
b: ['SALSDLHAHKLRVDPVNFKLLSHC', 'KEFQLSAQLCERLGKIRDAAYFYK', 'KKLDLNCDGQLDFQEFLNLIGGLA', 'LSISADIETIGEILKKIIPTLEEG', 'DAVAKASKDTHVMDYRALVHERDE', 'KKADEFPDFYDSEEQMGPHQEAND', 'ASSAQRKRMSPKPELTEDQKQEIR', 'KKLDEQYQKYKFMELNLAQKKRRL', 'PVEGAFTVFSDFLTYKSGVYKHEA', 'LLSSEATVRSLQAAVDQRDQMIQQ', 'KAAGWSELSERDAIYKEFSFKNFN', 'PVFEQAQYRETLRENVEEGYPILQ', 'VPENIPAGSSIFKVQAEDKDTGSG', 'ASHLGSELGGTEELCCAAELDNPM', 'LSEGLTTLDTVVSSSCCTSLDYMV', 'KKETSQIDGPSLNNTYGFKVSIQN', 'DIPVVCLESDNGNIMIQKHDGITV', 'LLENMTEVVRKGIQEAQVQLQKAN', 'KKDRESGEVFSIIQEYKPKDVTRA', 'GEEGGAAGLWVRSGAAAAAGAGGG', 'LLSSSYPEDFEKLLRQEEDSYNRE', 'KTFSHELSDFGLESTTGEVPVVAI', 'IQTESPQEGIVTWKTYHTYIKASG', 'KKDDCISKFKTLNDENSSHLLYFN', 'TVVTEAGNLLKDNATQEEILHYLE', 'KHAEGQLSFIQHSLINPEVFCFAD', 'PVEDSQGQTKVGNEKWTEGGGHSR', 'PVEISFCVGYVAVMSDLEVLLLKL', 'VTHQDIRSQTPLLIRSGITGLEVT',

In [3]:
# load the values
import json
all_results = json.load(open(OUTPUT_DIRECTORY + 'summary.json', 'r'))

In [4]:
for i, entry in all_results.items():
    print(f'---------- Alignments for spectrum {i} ----------')
    for a in entry['alignments']:
        print(f'{a["sequence"]} \t b score: {a["b_score"]} \t y score: {a["y_score"]}')

---------- Alignments for spectrum 0 ----------
LPVNSPMTKGD 	 b score: 0 	 y score: 0
---------- Alignments for spectrum 1 ----------
DAAIVGYK 	 b score: 3 	 y score: 6
AAIVGYKD 	 b score: 0 	 y score: 0
---------- Alignments for spectrum 2 ----------
---------- Alignments for spectrum 3 ----------
DPQVEQLEL 	 b score: 6 	 y score: 2
---------- Alignments for spectrum 4 ----------
---------- Alignments for spectrum 5 ----------
---------- Alignments for spectrum 6 ----------
ELTNIELL 	 b score: 6 	 y score: 1
KTELTNIE 	 b score: 0 	 y score: 1
---------- Alignments for spectrum 7 ----------
---------- Alignments for spectrum 8 ----------
